# **DeepHeartNet: Deep 1D-ResNet Framework for Emotion Recognition from Heart Rate Signals**

# ================================================
# 1. SETUP: Colab Pro+, Kaggle, and Libraries
# ================================================

In [ ]:

!pip install -U kaggle wfdb torch torchvision torchaudio scikit-learn pyhrv

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Set A100 GPU as device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 127.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.3 MB/s eta 0:00:00
  

# ================================================
# 2. Download Dataset from Kaggle
# ================================================

In [ ]:

from google.colab import files
files.upload()  # Upload your kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d isameeramohamed/emotions-and-heart-rate-scale-classification
!unzip -o emotions-and-heart-rate-scale-classification.zip -d ./hr_dataset

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/isameeramohamed/emotions-and-heart-rate-scale-classification
License(s): unknown
  0% 0.00/184k [00:00<?, ?B/s]
100% 184k/184k [00:00<00:00, 604MB/s]
Archive:  emotions-and-heart-rate-scale-classification.zip
  inflating: ./hr_dataset/heart_rate_emotion_dataset.csv  


# ================================================
# 3. Load Data (assume .csv)
# ================================================

In [ ]:

df = pd.read_csv('/content/hr_dataset/heart_rate_emotion_dataset.csv') # Change file name as per actual
print(df.head())
df.info()



   HeartRate  Emotion
0         65      sad
1         79  neutral
2         73  neutral
3        100    happy
4         99    angry
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   HeartRate  100000 non-null  int64 
 1   Emotion    100000 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


# ================================================
# 4. Feature Extraction and Preparation: BPM, HRV (Time/Freq domain)
# ================================================


In [ ]:

from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 1. Feature extraction: mean, std, min, max (trivial with single value, but can be extended if windowing)
def extract_features(hr):
    features = {}
    features['mean_hr'] = hr
    features['std_hr'] = 0
    features['min_hr'] = hr
    features['max_hr'] = hr
    return features

# Build feature dataframe
feature_rows = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    feats = extract_features(row['HeartRate'])
    feats['emotion'] = row['Emotion']
    feature_rows.append(feats)
feature_df = pd.DataFrame(feature_rows)
print(feature_df.head())

100%|██████████| 100000/100000 [00:04<00:00, 24840.85it/s]


   mean_hr  std_hr  min_hr  max_hr  emotion
0       65       0      65      65      sad
1       79       0      79      79  neutral
2       73       0      73      73  neutral
3      100       0     100     100    happy
4       99       0      99      99    angry


In [ ]:
# 2. Label binarization (example: "happy"/"angry" = 1, others = 0)
# Adjust mapping as per your classification target
positive_emotions = ['happy', 'excited', 'joy']  # example positive labels
feature_df['label_bin'] = feature_df['emotion'].apply(lambda x: 1 if x.lower() in positive_emotions else 0)

In [ ]:
# 3. Train/Validation Split
X = feature_df.drop(['emotion', 'label_bin'], axis=1).values
y = feature_df['label_bin'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print('X_train shape:', X_train.shape)
print('y_train distribution:', np.bincount(y_train))

# ================================================
# 5. PyTorch Dataset and DataLoader
# ================================================

In [ ]:



class HRDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = HRDataset(X_train, y_train)
test_ds = HRDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)

X_train shape: (80000, 4)
y_train distribution: [68574 11426]


# ================================================
# 6. Deep Classifier: "ResNet18" + Dense Layers
# (For tabular, use 1D Conv as ResNet block)
# ================================================

In [ ]:

class BasicBlock1D(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_planes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm1d(planes)
            )
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet18_1D(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.layer1 = BasicBlock1D(in_channels, 32)
        self.layer2 = BasicBlock1D(32, 64, stride=2)
        self.layer3 = BasicBlock1D(64, 128, stride=2)
        self.layer4 = BasicBlock1D(128, 256, stride=2)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = x.unsqueeze(1)  # [B, 1, features]
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.gap(x).squeeze(-1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

num_classes = 2
model = ResNet18_1D(in_channels=1, num_classes=num_classes).to(device)

# ================================================
# 7. Training Loop
# ================================================

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 20

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(train_loader):.4f}")



Epoch 1/20 - Loss: 0.1818
Epoch 2/20 - Loss: 0.1787
Epoch 3/20 - Loss: 0.1769
Epoch 4/20 - Loss: 0.1761
Epoch 5/20 - Loss: 0.1747
Epoch 6/20 - Loss: 0.1740
Epoch 7/20 - Loss: 0.1746
Epoch 8/20 - Loss: 0.1736
Epoch 9/20 - Loss: 0.1728
Epoch 10/20 - Loss: 0.1737
Epoch 11/20 - Loss: 0.1707
Epoch 12/20 - Loss: 0.1710
Epoch 13/20 - Loss: 0.1711
Epoch 14/20 - Loss: 0.1691
Epoch 15/20 - Loss: 0.1680
Epoch 16/20 - Loss: 0.1683
Epoch 17/20 - Loss: 0.1673
Epoch 18/20 - Loss: 0.1668
Epoch 19/20 - Loss: 0.1658
Epoch 20/20 - Loss: 0.1655


# ================================================
# 8. Evaluation
# ================================================

In [ ]:
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_auc_score,
    matthews_corrcoef
)
import numpy as np

model.eval()
all_preds, all_trues, all_probs = [], [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        probs = torch.softmax(out, dim=1)
        preds = torch.argmax(probs, 1).cpu().numpy()
        all_preds.extend(list(preds))
        all_trues.extend(list(yb.cpu().numpy()))
        all_probs.extend(probs.cpu().numpy())

# Convert to numpy arrays
all_trues = np.array(all_trues)
all_preds = np.array(all_preds)
all_probs = np.array(all_probs)

acc = accuracy_score(all_trues, all_preds)
f1 = f1_score(all_trues, all_preds, average='weighted')
precision = precision_score(all_trues, all_preds, average='weighted')
recall = recall_score(all_trues, all_preds, average='weighted')
mcc = matthews_corrcoef(all_trues, all_preds)
cm = confusion_matrix(all_trues, all_preds)
cr = classification_report(all_trues, all_preds)
# For binary ROC-AUC
if len(np.unique(all_trues)) == 2:
    auc = roc_auc_score(all_trues, all_probs[:,1])
else:  # multiclass
    auc = roc_auc_score(all_trues, all_probs, multi_class='ovr', average='macro')

print(f"Test Accuracy: {acc:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")
print(f"Precision (weighted): {precision:.4f}")
print(f"Recall (weighted): {recall:.4f}")
print(f"Matthews Correlation Coefficient: {mcc:.4f}")
print(f"ROC-AUC: {auc:.4f}")
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", cr)


Test Accuracy: 0.9208
F1-Score (weighted): 0.9131
Precision (weighted): 0.9169
Recall (weighted): 0.9208
Matthews Correlation Coefficient: 0.6391
ROC-AUC: 0.9553
Confusion Matrix:
 [[16878   265]
 [ 1318  1539]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.96     17143
           1       0.85      0.54      0.66      2857

    accuracy                           0.92     20000
   macro avg       0.89      0.76      0.81     20000
weighted avg       0.92      0.92      0.91     20000



thank you